In [1]:
%matplotlib qt4
from __future__ import division

from models import tools, optimize, models, filters
from models.tests import PerformanceTest

import numpy as np
import pandas as pd
import sklearn as sk
import matplotlib.pyplot as plt

In [15]:
data = tools.load_data(offset=1700000, limit=500000)
data = data[filters.usa_states(data)]
#data = tools.unknown_answers(data)
print len(data)

31006


In [3]:
elo_test = PerformanceTest(models.EloModel(), data, split_data=True)
elo_test.run()
elo_test.results['train']

RMSE: 0.401072157963
AUC: 0.761769439159
OFF: -0.00320089485118
CORRECT: 1987
ACCURACY: 0.771650485437
Set Size: 2575

In [5]:
elot_test = PerformanceTest(models.EloResponseTime(), data, split_data=True)
elot_test.run()
elot_test.results['train']

RMSE: 0.419755451333
AUC: 0.734505173408
OFF: -0.0359641950947
CORRECT: 17177
ACCURACY: 0.737052134735
Set Size: 23305

In [16]:
pfa_test = PerformanceTest(models.PFAModel(models.EloModel()), data)
pfa_test.run()
pfa_test.results['train']

RMSE: 0.378762640406
AUC: 0.795581771756
OFF: 0.0129000311924
CORRECT: 24584
ACCURACY: 0.792878797652
Set Size: 31006

In [18]:
pfag_test = PerformanceTest(models.PFAGong(models.EloModel(), gamma=1.5, delta=-0.2, decay=0.8), data)
pfag_test.run()
pfag_test.results['train']

RMSE: 0.383440850163
AUC: 0.787201009454
OFF: 0.0250953456438
CORRECT: 24499
ACCURACY: 0.790137392763
Set Size: 31006

In [19]:
pfaf_test = PerformanceTest(models.PFAForgetting(models.EloModel()), data)
pfaf_test.run()
pfaf_test.results['train']

RMSE: 0.384352375631
AUC: 0.770040105839
OFF: 0.0441200371592
CORRECT: 24571
ACCURACY: 0.794021651317
Set Size: 30945

In [20]:
pfas_test = PerformanceTest(models.PFASpacing(models.EloModel(), iota=1.5, decay_rate=0.18), data)
pfas_test.run()
pfas_test.results['train']

RMSE: 0.377471537896
AUC: 0.79583708885
OFF: 0.0199368217543
CORRECT: 24598
ACCURACY: 0.793330323163
Set Size: 31006

In [3]:
pfast_test = PerformanceTest(models.PFAStaircase(models.EloModel(), staircase={(-np.inf, np.inf): 5}), data)
pfast_test.run()
pfast_test.results['train']

0.33809650378991235

In [15]:
fpr, tpr, thresholds = sk.metrics.roc_curve(
    pfag_test.train_values['observed'],
    pfag_test.train_values['predicted'], pos_label=1)
plt.plot(fpr, tpr)

In [18]:
positives = []
negatives = []
p = pfa_test.train_values[pfa_test.train_values['observed'] == 1]
n = pfa_test.train_values[pfa_test.train_values['observed'] == 0]

intervals = zip(np.arange(0, 0.99, 0.01), np.arange(0.01, 1, 0.01))
for lower, upper in intervals:
    positive_count = len(p[(p['predicted'] > lower) & (p['predicted'] < upper)])
    negative_count = len(n[(n['predicted'] > lower) & (n['predicted'] < upper)])
    positives.append(positive_count)
    negatives.append(negative_count)

In [19]:
plt.plot([np.mean(interval) for interval in intervals], positives, 'g-')
plt.plot([np.mean(interval) for interval in intervals], negatives, 'r-')